In [2]:
import numpy
import pandas as pd
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

C:\deeplearning\anaconda\envs\ai-projects\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# load the data
dataframe = pd.read_csv("housing.csv", sep=',', header=0) 
dataset = dataframe.values

In [4]:
dataframe.head()

,BIZPROP,ROOMS,AGE,HIGHWAYS,TAX,PTRATIO,LSTAT,VALUE
0,19.58,7.489,90.8,5,403,14.7,1.73,50.0
1,19.58,7.802,98.2,5,403,14.7,1.92,50.0
2,19.58,8.375,93.9,5,403,14.7,3.32,50.0
3,19.58,7.929,96.2,5,403,14.7,3.70,50.0
4,2.46,7.831,53.6,3,193,17.8,4.45,50.0


In [5]:
# split into input features and output target variable
features = dataset[:,0:7]
target = dataset[:,7]

In [6]:
# fix random seed for reproducibility
seed = 9
# Randomly divide the data into 10 subsets sets 
kfold = KFold(n_splits=10, random_state=seed)

In [7]:
def simple_shallow_seq_net():
    # create a sequential ANN
    model = Sequential()
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    sgd = optimizers.SGD(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [9]:
# fit the neural net model
estimator = KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100, batch_size=50, verbose=0)

In [10]:
# cross validate across the sub sets
results = cross_val_score(estimator, features, target, cv=kfold)
print("simple_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_shallow_seq_model:(163.41) MSE


In [11]:
# save the model for later use
estimator.fit(features, target)
estimator.model.save('simple_shallow_seq_net.h5')

In [12]:
# Standardise the data and fit the neural net model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)

In [14]:
# Train and evaluate
results = cross_val_score(pipeline, features, target, cv=kfold)
print("simple_std_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_std_shallow_seq_net:(65.55) MSE


In [15]:
def deep_seq_net():
    # create a sequential model
    model = Sequential()
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(7,activation='tanh'))
    model.add(Dense(7,activation='sigmoid'))
    model.add(Dense(7,activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal'))
    sgd = optimizers.SGD(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [16]:
# Standardise the data and fit the neural net model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=deep_seq_net, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)

In [17]:
# cross validate across the sub sets
results = cross_val_score(pipeline, features, target, cv=kfold)
print("deep_seq_net:(%.2f) MSE" % (results.std()))

deep_seq_net:(58.79) MSE


In [ ]:
# save the model for later use
pipeline.fit(features, target)
pipeline.named_steps['estimator'].model.save('deep_seq_net.h5')

In [19]:
def deep_and_wide_net():
    # create a sequential model
    model = Sequential()
    model.add(Dense(21, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(21,activation='relu'))
    model.add(Dense(21,activation='relu'))
    model.add(Dense(21,activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    sgd = optimizers.SGD(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=sgd) 
    return model

In [20]:
# Standardise the data and fit the neural net model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=deep_and_wide_net, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)

In [21]:
# cross validate across the sub sets
results = cross_val_score(pipeline, features, target, cv=kfold)
print("deep_and_wide_model:(%.2f) MSE" % (results.std()))

deep_and_wide_model:(34.43) MSE


In [22]:
# save the model for later use
pipeline.fit(features, target)
pipeline.named_steps['estimator'].model.save('deep_and_wide_net.h5')